# Data wrangling and datetimes

Dates and times are a special kind of data type. In this notebook, we will keep exploring the `orders` and `orderlines` datasets from Eniac and learn to deal with them.

In [26]:
import pandas as pd

url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' # orderlines.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


## Data exploration

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


Two variables need to be modified: 

* `unit_price`: it is detected as an object but it has to be a float. Why is that happening? 
* `date`: needs to be transformed to a date-time format. 

For the moment we will only focus on the `date`. Since it contains both the date and the time, we will transform the data type using the pandas method `.to_datetime()`: 

In [33]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


In [ ]:
df.describe()

,id,id_order,product_id,product_quantity
count,2.939830e+05,293983.000000,293983.0,293983.000000
mean,1.397918e+06,419999.116544,0.0,1.121126
std,1.530096e+05,66344.486479,0.0,3.396569
min,1.119109e+06,241319.000000,0.0,1.000000
25%,1.262542e+06,362258.500000,0.0,1.000000
50%,1.406940e+06,425956.000000,0.0,1.000000
75%,1.531322e+06,478657.000000,0.0,1.000000
max,1.650203e+06,527401.000000,0.0,999.000000


You can count the quantity of observations with `.value_counts()`.

In [ ]:
df['sku'].value_counts()

MIC0036      6282
APP1216      5627
APP0662      5445
APP1190      5039
APP0663      3942
             ... 
APP2110         1
TWS0053-A       1
APP1837         1
UAG0026         1
REP0299         1
Name: sku, Length: 7951, dtype: int64

**Exercise**: check whether the dates from the `orderlines` dataset match with the dates on the `orders` dataset.

In [21]:
# code here
import pandas as pd 
url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' # orderlines.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
order_lines = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1BLEHcP-9fm9Rv7A01H3co2XBMnSr66YC/view?usp=sharing' # orders.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)

In [22]:
order_lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [ ]:
order_lines['date'] = pd.to_datetime(order_lines['date'])
orders['created_date'] = pd.to_datetime(orders['created_date'])

In [15]:
merged_df = order_lines.merge(orders, how='left', left_on='id_order', right_on='order_id')

In [24]:
merged_df[merged_df['date'] == merged_df['created_date']].shape[0]

166451

In [17]:
 merged_df.shape[0]

293983

Here another way to solve the same problem

In [18]:
(
order_lines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .assign(check_date = lambda x: x['date'] == x['created_date'])
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,order_id,created_date,total_paid,state,check_date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,299539.0,2017-01-01 00:07:19,18.99,Shopping Basket,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,299540.0,2017-01-01 00:19:45,399.00,Shopping Basket,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,299541.0,2017-01-01 00:20:57,474.05,Shopping Basket,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,299542.0,2017-01-01 00:51:40,68.39,Shopping Basket,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,299543.0,2017-01-01 01:06:38,23.74,Shopping Basket,True
...,...,...,...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,527398.0,2018-03-14 13:57:25,42.99,Shopping Basket,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,527399.0,2018-03-14 13:57:34,141.58,Shopping Basket,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,527400.0,2018-03-14 13:57:41,19.98,Shopping Basket,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,527388.0,2018-03-14 13:51:59,34.98,Shopping Basket,False


In [ ]:
ol_order_info = order_lines.groupby('id_order').agg({'date':'min'}).reset_index()
ol_order_info

,id_order,date
0,241319,2017-01-02 12:26:59
1,241355,2017-01-20 14:07:17
2,241423,2017-11-06 12:47:20
3,242832,2017-12-31 17:26:40
4,243330,2017-02-15 17:07:44
...,...,...
204850,527397,2018-03-14 13:56:38
204851,527398,2018-03-14 13:57:25
204852,527399,2018-03-14 13:57:34
204853,527400,2018-03-14 13:57:41


In [ ]:
(
    ol_order_info
 .merge(orders, how='inner', left_on='id_order', right_on='order_id')
 .assign(check_date = lambda x: x['date'] == x['created_date'])
    )

,id_order,date,order_id,created_date,total_paid,state,check_date
0,241319,2017-01-02 12:26:59,241319,2017-01-02 13:35:40,44.99,Cancelled,False
1,241423,2017-11-06 12:47:20,241423,2017-11-06 13:10:02,136.15,Completed,False
2,242832,2017-12-31 17:26:40,242832,2017-12-31 17:40:03,15.76,Completed,False
3,243330,2017-02-15 17:07:44,243330,2017-02-16 10:59:38,84.98,Completed,False
4,243784,2017-11-24 13:27:41,243784,2017-11-24 13:35:19,157.86,Cancelled,False
...,...,...,...,...,...,...,...
204691,527397,2018-03-14 13:56:38,527397,2018-03-14 13:56:38,42.99,Place Order,True
204692,527398,2018-03-14 13:57:25,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204693,527399,2018-03-14 13:57:34,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204694,527400,2018-03-14 13:57:41,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


In [ ]:
orders.sort_values('created_date', ascending=False).head(10)

,order_id,created_date,total_paid,state
226908,527401,2018-03-14 13:58:36,18.98,Place Order
226898,527391,2018-03-14 13:57:54,54.98,Pending
226907,527400,2018-03-14 13:57:41,19.98,Shopping Basket
226906,527399,2018-03-14 13:57:34,141.58,Shopping Basket
226905,527398,2018-03-14 13:57:25,42.99,Shopping Basket
226904,527397,2018-03-14 13:56:38,42.99,Place Order
226903,527396,2018-03-14 13:55:44,141.58,Place Order
152743,452946,2018-03-14 13:55:31,3184.96,Pending
226902,527395,2018-03-14 13:55:22,20.99,Shopping Basket
226901,527394,2018-03-14 13:55:15,161.00,Place Order


## Aggregating with pandas

* Grouping and aggregating is one of the main ways to explore data. The main tools to do that with pandas are:
    * [`pandas.DataFrame.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html).
    * [`pandas.DataFrame.agg()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html?highlight=agg#pandas.DataFrame.agg) 
        * [`pandas.DataFrame.count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)

In [ ]:
df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


How many products do we have on each sku?

In [ ]:
df.groupby(['sku']).agg({'product_quantity':'count'})

,product_quantity
sku,
8MO0001-A,2
8MO0003-A,3
8MO0007,29
8MO0008,30
8MO0009,28
...,...
ZAG0041,2
ZAG0042,1
ZEP0007,5


Different levels of aggregation answer different business questions:

- **count**: How many times has each product (identified by its `sku`) been purchased in an order? 

- **sum**: How many units of each product have been sold overall? 

- **mean**: How many units/order are sold for each product?

In [ ]:
df.groupby(['sku']).agg({'product_quantity':['count', 'sum', 'mean']})

product_quantity              
                     count sum      mean
sku                                     
8MO0001-A                2   2  1.000000
8MO0003-A                3   3  1.000000
8MO0007                 29  30  1.034483
8MO0008                 30  31  1.033333
8MO0009                 28  30  1.071429
...                    ...  ..       ...
ZAG0041                  2   2  1.000000
ZAG0042                  1   1  1.000000
ZEP0007                  5   5  1.000000
ZEP0008                  1   1  1.000000
par0072                 12  13  1.083333

[7951 rows x 3 columns]

`groupby` can also be combined with other pandas functions to analyse in more depth the datasets.

In [ ]:
df.groupby('sku')[['sku','product_quantity']].describe()

product_quantity                                             
                     count      mean       std  min  25%  50%  75%  max
sku                                                                    
8MO0001-A              2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
8MO0003-A              3.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
8MO0007               29.0  1.034483  0.185695  1.0  1.0  1.0  1.0  2.0
8MO0008               30.0  1.033333  0.182574  1.0  1.0  1.0  1.0  2.0
8MO0009               28.0  1.071429  0.262265  1.0  1.0  1.0  1.0  2.0
...                    ...       ...       ...  ...  ...  ...  ...  ...
ZAG0041                2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
ZAG0042                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
ZEP0007                5.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
ZEP0008                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
par0072               12.0  1.083333  0.288675  1.0  1.0  1.0  1.0  2.0

[7951 rows x 8 columns]

Now let's see how can we combine `.groupby()` and `.aggragate()` with `.sort_values()`.

I would like to see the top 10 most sold products on our data (total product quantity). 

In [ ]:
(
df
    .groupby('sku')['product_quantity']
    .sum()
    .sort_values(ascending=False).head(10)
)

sku
APP1190    6366
MIC0036    6316
APP1216    5648
APP0662    5487
APP0663    4164
MMW0016    2615
APP0698    2348
SAT0054    2322
APP1214    1985
WDT0183    1978
Name: product_quantity, dtype: int64

How can we sort values with multiple aggregated functions?

In [ ]:
(
df
    .groupby('sku')
    .agg({'product_quantity':['sum','count','std','mean','median']})
    .sort_values(('product_quantity','mean'), ascending=False)
)

product_quantity                                    
                       sum count         std       mean median
sku                                                           
APP1917                 32     1         NaN  32.000000   32.0
ADN0039                132     7   47.245559  18.857143    1.0
KIN0137                862    55  107.718263  15.672727    1.0
EVU0013                177    12   47.005077  14.750000    1.0
SEV0028               1122   122   90.353268   9.196721    1.0
...                    ...   ...         ...        ...    ...
APP1546                  1     1         NaN   1.000000    1.0
APP1546-A                4     4    0.000000   1.000000    1.0
MAK0009                  2     2    0.000000   1.000000    1.0
MAK0008                  1     1         NaN   1.000000    1.0
8MO0001-A                2     2    0.000000   1.000000    1.0

[7951 rows x 5 columns]

## Working with date time

We will start by creating a weekday column. You will have to combine two functions, [`pandas.DataFrame.assign()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html) from `pandas` and [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) from `datetime`.

First, let's talk about `.assign()`. It is a useful method for creating new columns:

In [ ]:
df.assign(new_colum = 'hi! I am a new column!').head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,new_colum
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,hi! I am a new column!
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,hi! I am a new column!
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,hi! I am a new column!
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,hi! I am a new column!
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,hi! I am a new column!


A new column often is the result of an operation between other columns in the dataframe:

In [ ]:
df.assign(total_price = df['product_quantity'] * df['unit_price']).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74


In Pandas, we can extract strings containing elements like the day of the month, the day of the week, the hour of the day... from `datetime` objects. We do so by using `dt.strftime()` in combination with `strftime` syntax. Find the cheat sheet for this syntax [here](https://strftime.org/). 

You'll understand it better with an example. Let's create the weekday column:

In [ ]:
df.assign(week_day = df['date'].dt.strftime('%A')).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,week_day
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,Sunday
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,Sunday
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,Sunday
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,Sunday
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,Sunday


In [ ]:
df['date'].dt.strftime('%B')

0         January
1         January
2         January
3         January
4         January
           ...   
293978      March
293979      March
293980      March
293981      March
293982      March
Name: date, Length: 293983, dtype: object

Now we can sort weekdays by amount of sales:

In [ ]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby('week_day')
    .agg({'product_quantity':['sum','count']})
    .sort_values(('product_quantity','sum'), ascending=False)
)

product_quantity       
                       sum  count
week_day                         
Monday               57624  50307
Wednesday            54131  47550
Tuesday              50871  44498
Friday               49566  44027
Thursday             48431  43634
Sunday               35667  32857
Saturday             33302  31110

Let's find out the top 1 product (in product quantity) by sku for each weekday:

In [ ]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby(['week_day','sku'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .reset_index()
    .groupby('week_day')
    .head(1)
)

,week_day,sku,product_quantity
0,Friday,APP1190,1827
1,Thursday,MIC0036,1389
3,Tuesday,MIC0036,1213
6,Wednesday,SEV0028,1014
10,Monday,APP0663,840
14,Saturday,MIC0036,806
23,Sunday,APP1190,692


In [ ]:
df.date

0        2017-01-01 00:07:19
1        2017-01-01 00:19:45
2        2017-01-01 00:20:57
3        2017-01-01 00:51:40
4        2017-01-01 01:06:38
                 ...        
293978   2018-03-14 13:57:25
293979   2018-03-14 13:57:34
293980   2018-03-14 13:57:41
293981   2018-03-14 13:58:01
293982   2018-03-14 13:58:36
Name: date, Length: 293983, dtype: datetime64[ns]

When using `pd.to_datetime` to convert "objects" like "28-07-1990 03:30:15" to a proper data type, you have captured both the date and the time. Sometimes, you might only need the date (without the time). An easy way to do this is with `dt.date`:

In [31]:
df.date.dt.date.head(3)

AttributeError: ignored

Note that these are not actually strings, even thoguht the `dtype` that pandas displays for the whole column is `object`:

In [ ]:
df.date.dt.date[0]

datetime.date(2017, 1, 1)

In [ ]:
type(df.date.dt.date[0])

datetime.date

You can create these `datetime.date` objects from strings using the module `datetime`. This module has a handful of useful functions for dates and times, [check its documentation here](https://docs.python.org/3/library/datetime.html#date-objects).

In [36]:
import datetime

datetime.date.fromisoformat("1990-07-28")

datetime.date(1990, 7, 28)

What we've outlined here is going to become useful and relevant whenever filtering a DataFrame by date, as you will see in the following challenges:

# CHALLENGES

**1. Which are the top 6 orders with the highest number of products sold?**

Using 'count'

In [27]:
(
df
    .groupby('id_order')
    .agg({'product_quantity':'count'})
    .sort_values('product_quantity', ascending=False)
    .head(6)
)

,product_quantity
id_order,
395611,256
484334,140
301934,131
329309,73
349364,72
375621,70


**2. Which are the top 6 orders for the 1 of July of 2017 with the highest number of products sold ?**

Combining `.assign()` with `.dt.date` will help you extract the date of a datetime column and use this information to filter it. Use the `datetime` module and its `date.fromisoformat()` method as we showed above.

Here is an example how to filter the first of January 2017.

In [38]:
#df['date'].dt.date
datetime.date.fromisoformat('2017-01-01')

datetime.date(2017, 1, 1)

In [39]:
df_date = df.assign(date = df['date'].dt.date).copy()
#example
df_date[df_date['date'] == datetime.date.fromisoformat('2017-01-01')]

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01
...,...,...,...,...,...,...,...
612,1120279,300119,0,1,JBL0107,94.99,2017-01-01
613,1120287,300126,0,1,LIF0112,61.74,2017-01-01
614,1120289,300127,0,1,EIZ0017,739.99,2017-01-01
615,1120290,300128,0,1,APP1195,1.193.99,2017-01-01


In [41]:
(
df_date
    .loc[df_date['date'] == datetime.date.fromisoformat('2017-07-01')]
    .groupby('id_order')
    .agg({'product_quantity':'count'})
    .sort_values('product_quantity', ascending=False)
    .head(6)
)

,product_quantity
id_order,
371217,6
371178,5
371285,5
371309,4
371120,4
371263,4


In [ ]:
df_date_filtered = df_date.loc[df_date['date'] == datetime.date.fromisoformat('2017-07-01')].copy()
(
    df_date_filtered.groupby('id_order')
                                   .agg({'product_quantity':'count'})
                                   .sort_values('product_quantity', ascending=False)
                                   .head(6)
)

**3. Which is the month with the highest number of units sold?** 

Remember to look at the documentation of [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) 

In [ ]:
(
df
    .assign(month = df['date'].dt.strftime('%B'))
    .groupby('month')
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .head(1)
)

,product_quantity
month,
January,66415


The results enormous differences between some months. Why do you think this is happening? Do we only have one year of data? In case you have multiple years, filter for only years that are complete to find out the best month for that year.

ATTENTION: the output of the function `.dt.strftime()` is a string!

In [ ]:
new=df.assign(month = df['date'].dt.strftime('%m'))
new=new.assign(year=df['date'].dt.strftime('%y'))
new.groupby("year").agg({"month":"unique"})

,month
year,
17,"[01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12]"
18,"[01, 02, 03]"


In [46]:
df_results = (
df
    .assign(month = df['date'].dt.strftime('%B'),
           year = df['date'].dt.strftime('%Y'))
   
    .groupby(['year', 'month'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
)

In [50]:
df_results.reset_index(inplace=True)

In [53]:
df_results[df_results.year == '2018'].sort_values('product_quantity', ascending=False).head(1)

,year,month,product_quantity
1,2018,January,41950


In [58]:
#df_results.groupby(['year'])['month'].agg({'product_quantity': 'max'})

In [ ]:
df = df.assign(week_day = df['date'].dt.strftime('%A'), month = df["date"].dt.strftime("%B"), year = df["date"].dt.strftime("%Y"))

**4. For each month, find the day of the week with the highest amount of products sold** 

See below an example of the expected output.

In [72]:
df_agg = (
df
    .assign(month = df['date'].dt.strftime('%B'),
           year = df['date'].dt.strftime('%Y'), 
           week_day = df['date'].dt.strftime('%A'))
    .query('year == "2017"')
    .groupby(['month','week_day'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    #.reset_index()
    #.groupby('month')
    .head(10)
)

In [71]:
df_agg

,month,week_day,id,id_order,product_id,product_quantity
0,November,Friday,12664731298,3855230755,0,9717
1,November,Thursday,10043885361,3053872553,0,7727
2,November,Wednesday,9701037307,2957310683,0,7137
3,November,Monday,9233723837,2820193980,0,7109
4,December,Wednesday,9846380911,3047854704,0,7010
5,December,Thursday,8239961080,2552557668,0,5932
6,December,Tuesday,7565616309,2341078108,0,5459
7,January,Monday,5352260701,1443885379,0,5420
8,December,Friday,7416004242,2296906169,0,5373
9,December,Monday,7236057104,2234872494,0,5181


In [66]:
df_agg.reset_index(inplace=True)

In [67]:
df_agg['month']

0    November
1    November
2    November
3    November
4    December
5    December
6    December
7     January
8    December
9    December
Name: month, dtype: object

In [ ]:
(
    df.assign(day = df['date'].dt.strftime('%B, %A, %Y')) #%D%x 
          .groupby(['day'])
          .agg({'product_quantity':'sum'})
          .sort_values(('product_quantity'), ascending=False)
          .reset_index().head(10)
)

,day,product_quantity
0,"November, Friday, 2017",9717
1,"January, Tuesday, 2018",8403
2,"January, Monday, 2018",8327
3,"November, Thursday, 2017",7727
4,"November, Wednesday, 2017",7137
5,"November, Monday, 2017",7109
6,"December, Wednesday, 2017",7010
7,"January, Thursday, 2018",6952
8,"January, Wednesday, 2018",6844
9,"December, Thursday, 2017",5932


In [ ]:
(
df
    .assign(year = df['date'].dt.strftime('%Y'),
            month = df.date.dt.strftime('%B'), 
            week_day = df.date.dt.strftime('%A')) # dt.month)
    .query("year == '2017'")
    .groupby(['month', 'week_day'])
    .agg({'product_quantity': 'sum'})
    .reset_index()
#     .query('month == "November"')    
    .groupby('month')['product_quantity']
    .max()
)

month
April        3286
August       2634
December     7010
February     3066
January      5420
July         4018
June         2572
March        2628
May          3574
November     9717
October      3849
September    2893
Name: product_quantity, dtype: int64

In [ ]:
week_days = (df
    .assign(year = df['date'].dt.strftime('%Y'),
            month = df.date.dt.strftime('%B'), 
            week_day = df.date.dt.strftime('%A')))['week_day'].unique()

In [ ]:
week_days

array(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday'], dtype=object)

In [ ]:
max(week_days)

'Wednesday'